In [66]:
!pip install azure-storage-blob # Microsoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

In [67]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar

In [68]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [69]:
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

In [70]:
import json
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO

# Specify the path to your JSON configuration file
config_file_path = 'config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

# Connection string from configuration file
CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]

# Name of the container
CONTAINER_AZURE = 'realestatesales'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get the container client
container_client = blob_service_client.get_container_client(CONTAINER_AZURE)

# Initialize an empty DataFrame to collect data
REsales_df = pd.DataFrame()

# List all blobs in the specified container and process each one
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(f"Processing blob: {blob.name}")
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')

    # Create a DataFrame from the CSV content
    REsales_df = pd.read_csv(StringIO(blob_content))


REsales_df = REsales_df.copy()

# Display the shape and head of the DataFrame
REsales_df.shape
REsales_df.head(10)

Processing blob: realestatesales.csv


<ipython-input-70-53223a14c60c>:37: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  REsales_df = pd.read_csv(StringIO(blob_content))


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.00,248400.00,0.54,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.00,239900.00,0.46,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.00,325000.00,0.46,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.00,202500.00,0.63,Residential,Two Family,NaN,NaN,NaN,NaN
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.00,400000.00,0.54,Residential,Single Family,NaN,NaN,NaN,NaN
5,200121,2020,12/15/2020,Avon,63 NORTHGATE,528490.00,775000.00,0.68,Residential,Single Family,NaN,NaN,NaN,POINT (-72.89675 41.79445)
6,20058,2020,06/01/2021,Barkhamsted,46 RATLUM MTN RD,203530.00,415000.00,0.49,Residential,Single Family,NaN,"2003 COLONIAL, 2140 SFLA, 2.99 AC",NaN,NaN
7,200046,2020,01/25/2021,Beacon Falls,34 LASKY ROAD,158030.00,243000.00,0.65,Residential,Single Family,NaN,NaN,NaN,NaN
8,200016,2020,11/13/2020,Beacon Falls,9 AVON COURT,65590.00,100000.00,0.66,Residential,Condo,NaN,NaN,NaN,NaN
9,2020360,2020,08/10/2021,Berlin,94 PERCIVAL AVE,140600.00,190790.00,0.74,Residential,Single Family,NaN,NaN,NaN,NaN


In [71]:
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1054159 non-null  int64  
 1   List Year         1054159 non-null  int64  
 2   Date Recorded     1054157 non-null  object 
 3   Town              1054159 non-null  object 
 4   Address           1054108 non-null  object 
 5   Assessed Value    1054159 non-null  float64
 6   Sale Amount       1054159 non-null  float64
 7   Sales Ratio       1054159 non-null  float64
 8   Property Type     671713 non-null   object 
 9   Residential Type  660275 non-null   object 
 10  Non Use Code      302242 non-null   object 
 11  Assessor Remarks  161472 non-null   object 
 12  OPM remarks       11564 non-null    object 
 13  Location          254643 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 112.6+ MB


In [72]:
# After running REsales_df.info(), we see that the columns "Non Use Code", "Assessor Remarks", "OPM Remarks", and
# "Location" are more than 70% null, so for the purposes of our assignment I will drop these columns entirely. In
# addition, "List Year" is not relevant to us and will potentially impede the process of formatting the date, as
# the list year and "Date Recorded" years often differ, so this column will also be dropped.
REsales_df.drop(columns=['List Year', 'Non Use Code', 'Assessor Remarks', 'OPM remarks', 'Location'], inplace=True)
REsales_df.columns
REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020177,04/14/2021,Ansonia,323 BEAVER ST,133000.00,248400.00,0.54,Residential,Single Family
1,2020225,05/26/2021,Ansonia,152 JACKSON ST,110500.00,239900.00,0.46,Residential,Three Family
2,2020348,09/13/2021,Ansonia,230 WAKELEE AVE,150500.00,325000.00,0.46,Commercial,NaN
3,2020090,12/14/2020,Ansonia,57 PLATT ST,127400.00,202500.00,0.63,Residential,Two Family
4,200500,09/07/2021,Avon,245 NEW ROAD,217640.00,400000.00,0.54,Residential,Single Family


In [73]:
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1054159 non-null  int64  
 1   Date Recorded     1054157 non-null  object 
 2   Town              1054159 non-null  object 
 3   Address           1054108 non-null  object 
 4   Assessed Value    1054159 non-null  float64
 5   Sale Amount       1054159 non-null  float64
 6   Sales Ratio       1054159 non-null  float64
 7   Property Type     671713 non-null   object 
 8   Residential Type  660275 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 72.4+ MB


In [74]:
# Now we will look at the specific datatypes and perform modifications, if needed
REsales_df.dtypes

Serial Number         int64
Date Recorded        object
Town                 object
Address              object
Assessed Value      float64
Sale Amount         float64
Sales Ratio         float64
Property Type        object
Residential Type     object
dtype: object

In [75]:
# Let's convert "Date Recorded" from object to datetime
REsales_df['Date Recorded'] = pd.to_datetime(REsales_df['Date Recorded'], errors='coerce')

In [76]:
# Success
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Serial Number     1054159 non-null  int64         
 1   Date Recorded     1054157 non-null  datetime64[ns]
 2   Town              1054159 non-null  object        
 3   Address           1054108 non-null  object        
 4   Assessed Value    1054159 non-null  float64       
 5   Sale Amount       1054159 non-null  float64       
 6   Sales Ratio       1054159 non-null  float64       
 7   Property Type     671713 non-null   object        
 8   Residential Type  660275 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 72.4+ MB


In [77]:
REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020177,2021-04-14,Ansonia,323 BEAVER ST,133000.00,248400.00,0.54,Residential,Single Family
1,2020225,2021-05-26,Ansonia,152 JACKSON ST,110500.00,239900.00,0.46,Residential,Three Family
2,2020348,2021-09-13,Ansonia,230 WAKELEE AVE,150500.00,325000.00,0.46,Commercial,NaN
3,2020090,2020-12-14,Ansonia,57 PLATT ST,127400.00,202500.00,0.63,Residential,Two Family
4,200500,2021-09-07,Avon,245 NEW ROAD,217640.00,400000.00,0.54,Residential,Single Family


In [78]:
# I see there are several nulls in "Date Recorded" and "Address". Since there are only a few instances of
# these nulls, I will outright remove those specific rows.

REsales_df = REsales_df.dropna(subset=['Date Recorded'])
REsales_df = REsales_df.dropna(subset=['Address'])
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1054108 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   Serial Number     1054108 non-null  int64         
 1   Date Recorded     1054108 non-null  datetime64[ns]
 2   Town              1054108 non-null  object        
 3   Address           1054108 non-null  object        
 4   Assessed Value    1054108 non-null  float64       
 5   Sale Amount       1054108 non-null  float64       
 6   Sales Ratio       1054108 non-null  float64       
 7   Property Type     671709 non-null   object        
 8   Residential Type  660271 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 80.4+ MB


In [79]:
# "Property Type" and "Residential Type" both have about 400,000 null values which is potentially concerning.
# "Residential Type" only contains a value if "Property Type" is a residential address, so we can leave that as is.
# However, "Property Type" must contain a value. We can assume the nulls are vacant plots of land, but "Vacant Land"
# is already a value used for many of the rows. Thus, I will remove all the rows in which "Property Type" is blank.
REsales_df = REsales_df.dropna(subset=['Property Type'])
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 671709 entries, 0 to 1054158
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Serial Number     671709 non-null  int64         
 1   Date Recorded     671709 non-null  datetime64[ns]
 2   Town              671709 non-null  object        
 3   Address           671709 non-null  object        
 4   Assessed Value    671709 non-null  float64       
 5   Sale Amount       671709 non-null  float64       
 6   Sales Ratio       671709 non-null  float64       
 7   Property Type     671709 non-null  object        
 8   Residential Type  660271 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 51.2+ MB


In [80]:
REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020177,2021-04-14,Ansonia,323 BEAVER ST,133000.00,248400.00,0.54,Residential,Single Family
1,2020225,2021-05-26,Ansonia,152 JACKSON ST,110500.00,239900.00,0.46,Residential,Three Family
2,2020348,2021-09-13,Ansonia,230 WAKELEE AVE,150500.00,325000.00,0.46,Commercial,NaN
3,2020090,2020-12-14,Ansonia,57 PLATT ST,127400.00,202500.00,0.63,Residential,Two Family
4,200500,2021-09-07,Avon,245 NEW ROAD,217640.00,400000.00,0.54,Residential,Single Family


In [81]:
# Change "Assessed Value" and "Sale Amount" to 2 decimal places. We want to maintain the float datatype
# so we can conduct transformtion and loading later without issues, otherwise I would have added the
# dollar sign to the front of each value as well.
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', '{:.2f}'.format)

REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,2020177,2021-04-14,Ansonia,323 BEAVER ST,133000.00,248400.00,0.54,Residential,Single Family
1,2020225,2021-05-26,Ansonia,152 JACKSON ST,110500.00,239900.00,0.46,Residential,Three Family
2,2020348,2021-09-13,Ansonia,230 WAKELEE AVE,150500.00,325000.00,0.46,Commercial,NaN
3,2020090,2020-12-14,Ansonia,57 PLATT ST,127400.00,202500.00,0.63,Residential,Two Family
4,200500,2021-09-07,Avon,245 NEW ROAD,217640.00,400000.00,0.54,Residential,Single Family


In [82]:
# I want to sort the df by date recorded, in chronological order as opposed to town alphabetical order
REsales_df.sort_values(by='Date Recorded', ascending=True, inplace=True)

REsales_df.head(10)

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
857905,170177,1999-04-05,New London,184 PEQUOT AVE 309,46690.00,95000.00,0.49,Condo,Condo
878455,173165,2001-08-01,Shelton,19 BLACKBERRY LA,390810.00,635000.00,0.62,Single Family,Single Family
882230,173202,2001-08-23,Shelton,12 BOYSENBERRY LA,427910.00,558000.00,0.77,Single Family,Single Family
389697,60626,2001-09-04,Newington,110 MOYLAN CT,64750.00,88000.00,0.74,Condo,Condo
430234,60631,2001-09-05,Newington,48 EVERGREEN RD,166110.00,256000.00,0.65,Single Family,Single Family
410469,60643,2001-09-25,Naugatuck,217 OSBORN RD,183850.00,293711.00,0.63,Single Family,Single Family
420125,60072,2001-10-27,Greenwich,1010 NORTH ST,573930.00,660000.00,0.87,Single Family,Single Family
407724,60508,2003-07-17,Naugatuck,80 GORMAN ST,84200.00,158900.00,0.53,Single Family,Single Family
414022,60798,2004-06-05,Bristol,225 TYLER WAY,147320.00,329900.00,0.45,Single Family,Single Family
443581,60429,2004-07-27,South Windsor,305 MILL POND DR,113580.00,210000.00,0.54,Condo,Condo


In [83]:
# Now I will reset the indexes of the df
REsales_df.reset_index(drop=True, inplace=True)

# Running .head() to see the dates of the earliest 5 recorded real estate sales
REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,170177,1999-04-05,New London,184 PEQUOT AVE 309,46690.00,95000.00,0.49,Condo,Condo
1,173165,2001-08-01,Shelton,19 BLACKBERRY LA,390810.00,635000.00,0.62,Single Family,Single Family
2,173202,2001-08-23,Shelton,12 BOYSENBERRY LA,427910.00,558000.00,0.77,Single Family,Single Family
3,60626,2001-09-04,Newington,110 MOYLAN CT,64750.00,88000.00,0.74,Condo,Condo
4,60631,2001-09-05,Newington,48 EVERGREEN RD,166110.00,256000.00,0.65,Single Family,Single Family


In [84]:
# Running.tail() to see the dates of the latest 5 recorded real estate sales
REsales_df.tail()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
671704,210900024,2022-09-30,Woodbury,504 UPPER GRASSY HILL RD,391960.00,732000.00,0.54,Residential,Single Family
671705,212425,2022-09-30,Waterbury,72 WHEELER ST,55200.00,163000.00,0.34,Residential,Single Family
671706,210977,2022-09-30,Torrington,85 PULASKI ST,350000.00,450000.00,0.78,Commercial,NaN
671707,212411,2022-09-30,Waterbury,16 SAMUEL ST,86240.00,270000.00,0.32,Residential,Single Family
671708,211397,2022-09-30,Bristol,37 NORTON ST,102060.00,200000.00,0.51,Residential,Two Family


In [85]:
# After running REsales_df.head() and REsales_df.tail(), I see values that do not fit the context
# of my dataset. The data focuses on Connecticut real estate sales between 2001 and 2021, but
# the earliest record is in 1999 and the latest is in 2022. I will remove all the values that
# fall outside these bounds.

start_date = pd.to_datetime('2001-01-01')
end_date = pd.to_datetime('2021-12-31')

bounds = (REsales_df['Date Recorded'] >= start_date) & (REsales_df['Date Recorded'] <= end_date)

REsales_df = REsales_df[bounds]

REsales_df.head()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
1,173165,2001-08-01,Shelton,19 BLACKBERRY LA,390810.00,635000.00,0.62,Single Family,Single Family
2,173202,2001-08-23,Shelton,12 BOYSENBERRY LA,427910.00,558000.00,0.77,Single Family,Single Family
3,60626,2001-09-04,Newington,110 MOYLAN CT,64750.00,88000.00,0.74,Condo,Condo
4,60631,2001-09-05,Newington,48 EVERGREEN RD,166110.00,256000.00,0.65,Single Family,Single Family
5,60643,2001-09-25,Naugatuck,217 OSBORN RD,183850.00,293711.00,0.63,Single Family,Single Family


In [86]:
REsales_df.tail()

,Serial Number,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
630758,21082,2021-12-30,Wilton,69 WILTON CREST,254590.00,389000.00,0.65,Residential,Single Family
630759,21108,2021-12-30,Darien,20 HOPE DRIVE,1728160.00,3230000.00,0.54,Residential,Single Family
630760,210374,2021-12-31,West Hartford,645 PROSPECT AVENUE #7,202283.00,324500.00,0.62,Residential,Condo
630761,210375,2021-12-31,West Hartford,28 ARNOLD WAY 2A,87378.00,164900.00,0.53,Residential,Condo
630762,210013,2021-12-31,Morris,20 ESTHERS LN EXT,241920.00,436000.00,0.55,Residential,Single Family


In [87]:
# Check how many rows are left
REsales_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 630762 entries, 1 to 630762
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Serial Number     630762 non-null  int64         
 1   Date Recorded     630762 non-null  datetime64[ns]
 2   Town              630762 non-null  object        
 3   Address           630762 non-null  object        
 4   Assessed Value    630762 non-null  float64       
 5   Sale Amount       630762 non-null  float64       
 6   Sales Ratio       630762 non-null  float64       
 7   Property Type     630762 non-null  object        
 8   Residential Type  623382 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 48.1+ MB


In [88]:
# Finding the minimum assessed value
REsales_df["Assessed Value"].min()

0.0

In [89]:
# Finding the minimum sale amount
REsales_df["Sale Amount"].min()

0.0

In [90]:
# Finding the maximum assessed value
REsales_df["Assessed Value"].max()

881510000.0

In [91]:
# Finding the maximum sale amount
REsales_df["Sale Amount"].max()

5000000000.0

In [92]:
# Since none of the values are negative, it is safe to assume those values are fully cleaned

In [93]:
# Reverse the renaming of columns using the rename method with a dictionary
REsales_df = REsales_df.rename(columns={
    'Serial Number': 'fact_id',
    'Date Recorded': 'date_iso_format',
    'Town': 'town',
    'Address': 'address',
    'Assessed Value': 'assessed_value',
    'Sale Amount': 'sale_amount',
    'Sales Ratio': 'sales_ratio',
    'Property Type': 'property_type',
    'Residential Type': 'residential_type'
})

# This will revert the descriptive column names back to the original technical names in REsales_df


In [94]:
REsales_df.columns

Index(['fact_id', 'date_iso_format', 'town', 'address', 'assessed_value',
       'sale_amount', 'sales_ratio', 'property_type', 'residential_type'],
      dtype='object')

In [95]:
REsales_df.head()

,fact_id,date_iso_format,town,address,assessed_value,sale_amount,sales_ratio,property_type,residential_type
1,173165,2001-08-01,Shelton,19 BLACKBERRY LA,390810.00,635000.00,0.62,Single Family,Single Family
2,173202,2001-08-23,Shelton,12 BOYSENBERRY LA,427910.00,558000.00,0.77,Single Family,Single Family
3,60626,2001-09-04,Newington,110 MOYLAN CT,64750.00,88000.00,0.74,Condo,Condo
4,60631,2001-09-05,Newington,48 EVERGREEN RD,166110.00,256000.00,0.65,Single Family,Single Family
5,60643,2001-09-25,Naugatuck,217 OSBORN RD,183850.00,293711.00,0.63,Single Family,Single Family


In [96]:
## Creating the dimensions

In [97]:
# Create Date Dimension

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

start_date = pd.to_datetime('2001-08-01')
end_date = pd.to_datetime('2021-12-31')
# Create a DataFrame for the date dimension
dim_date = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})

dim_date.head(25)

# Extract attributes
dim_date['year_number'] = dim_date['date'].dt.year
dim_date['quarter_number'] = dim_date['date'].dt.quarter
dim_date['month_number'] = dim_date['date'].dt.month
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['day_number'] = dim_date['date'].dt.day
dim_date['day_name'] = dim_date['date'].dt.strftime('%A')
dim_date['date_iso_format'] = dim_date['date'].dt.strftime('%Y-%m-%d')
dim_date['date_id'] = dim_date['date'].dt.strftime('%Y%m%d')


dim_date['date_iso_format'] = dim_date['date'].dt.date
# Add week of the month and week of the year
dim_date['week_of_month'] = dim_date['date'].apply(week_of_month)
dim_date['week_of_year'] = dim_date['date'].dt.strftime('%U')

new_order = ['date_id', 'date_iso_format','year_number','month_number','day_number','quarter_number','month_name','day_name','week_of_year','week_of_month']
dim_date = dim_date[new_order]

dim_date.head(25)

,date_id,date_iso_format,year_number,month_number,day_number,quarter_number,month_name,day_name,week_of_year,week_of_month
0,20010801,2001-08-01,2001,8,1,3,August,Wednesday,30,1
1,20010802,2001-08-02,2001,8,2,3,August,Thursday,30,1
2,20010803,2001-08-03,2001,8,3,3,August,Friday,30,1
3,20010804,2001-08-04,2001,8,4,3,August,Saturday,30,1
4,20010805,2001-08-05,2001,8,5,3,August,Sunday,31,1
5,20010806,2001-08-06,2001,8,6,3,August,Monday,31,1
6,20010807,2001-08-07,2001,8,7,3,August,Tuesday,31,1
7,20010808,2001-08-08,2001,8,8,3,August,Wednesday,31,2
8,20010809,2001-08-09,2001,8,9,3,August,Thursday,31,2
9,20010810,2001-08-10,2001,8,10,3,August,Friday,31,2


In [98]:
## Create dim_location

# Assuming REsales_df is your main dataset with 'town' and 'address' columns
# Extract unique pairs of town and address
unique_pairs = REsales_df[['town', 'address']].drop_duplicates().reset_index(drop=True)

# Add a unique location_id starting from 1 to these unique pairs
unique_pairs['location_id'] = range(1, len(unique_pairs) + 1)

# Create dim_location with the desired column order
dim_location = unique_pairs[['location_id', 'town', 'address']]

# Display the dim_location DataFrame
print(dim_location)


        location_id           town            address
0                 1        Shelton   19 BLACKBERRY LA
1                 2        Shelton  12 BOYSENBERRY LA
2                 3      Newington      110 MOYLAN CT
3                 4      Newington    48 EVERGREEN RD
4                 5      Naugatuck      217 OSBORN RD
...             ...            ...                ...
517429       517430     Plainville     111 WHITING ST
517430       517431         Wilton    69 WILTON CREST
517431       517432         Darien      20 HOPE DRIVE
517432       517433  West Hartford   28 ARNOLD WAY 2A
517433       517434         Morris  20 ESTHERS LN EXT

[517434 rows x 3 columns]


In [101]:
# Create dim_propertyType

# Assuming REsales_df is your main dataset
# Extract unique property types
unique_property_types = pd.DataFrame(REsales_df['property_type'].unique(), columns=['property_type'])

# Add a unique property_type_id starting from 1 to these unique property types
unique_property_types['property_type_id'] = range(1, len(unique_property_types) + 1)

# Create dim_propertyType with the desired column order
dim_propertyType = unique_property_types[['property_type_id', 'property_type']]

# Display the dim_propertyType DataFrame
dim_propertyType.head(10)



,property_type_id,property_type
0,1,Single Family
1,2,Condo
2,3,Two Family
3,4,Three Family
4,5,Four Family
5,6,Residential
6,7,Commercial
7,8,Vacant Land
8,9,Apartments
9,10,Industrial


In [64]:
# Create dim_residentialType

# Assuming REsales_df is your main dataset
# Extract unique residential types
unique_residential_types = pd.DataFrame(REsales_df['residential_type'].unique(), columns=['residential_type'])

# Add a unique residential_type_id starting from 1 to these unique residential types
unique_residential_types['residential_type_id'] = range(1, len(unique_residential_types) + 1)

# Create dim_residentialType with the desired column order
dim_residentialType = unique_residential_types[['residential_type_id', 'residential_type']]

# Display the dim_residentialType DataFrame
dim_residentialType.head()

,residential_type_id,residential_type
0,1,Single Family
1,2,Condo
2,3,Two Family
3,4,Three Family
4,5,Four Family
